# Twitter Sentiment Analysis - POC
---

## 4. Cleaning Pipeline

This is an involved part and the beginning of the POC per se. For the final project, I will not be sampling the data. Here I'm using a very small sample so that I can quickly iterate and move foward with the project.


## POC Only - Sample Data

In [1]:
import os
import time

import numpy as np
import pandas as pd

# time notebook
start_notebook = time.time()

# load minimally prepared X, y train subsets
raw_path = os.path.join("..","data","1_raw","sentiment140")
X_train = pd.read_csv(os.path.join(raw_path, "X_train.csv"))
y_train = pd.read_csv(os.path.join(raw_path, "y_train.csv"))

In [2]:
# sample down considerably to X, y sample subsets
from sklearn.model_selection import train_test_split

X, X_rest, y, y_rest = train_test_split(X_train, y_train, test_size=0.9, random_state=158)

The plan is to forget about the `_rest` datasets and focus on the X, y small subsets, as if they were the entire training data.

In [3]:
print(f'Dataset size: {len(X):0.0f}')
print(f'Target distribution: {sum(y["target"])/len(y):0.3f}')

Dataset size: 119747
Target distribution: 0.501


In [4]:
# create an array of Tweets
X_array = np.array(X.iloc[:, 2]).ravel()
len(X_array)

119747

We lose the indices so we need to retain those.

In [5]:
X.insert(3, 'index', X.index)
X.index = range(len(X))

In [6]:
import re
import urlextract

from html import unescape
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from collections import Counter
from sklearn.base import BaseEstimator, TransformerMixin

def is_ascii(doc):
    try:
        doc.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    
url_extractor = urlextract.URLExtract()
lemmatizer = WordNetLemmatizer()
           
class DocumentToWordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, lower_case=True, replace_usernames=True,
                 unescape_html=True, replace_urls=True, 
                 replace_numbers=True, remove_junk=True, 
                 remove_punctuation=True, replace_emojis=True,
                 replace_nonascii=True, tokenize=True, 
                 remove_stopwords=True, lemmatization=True):
        self.lower_case = lower_case
        self.replace_usernames = replace_usernames
        self.unescape_html = unescape_html
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.remove_junk = remove_junk
        self.remove_punctuation = remove_punctuation
        self.replace_emojis = replace_emojis
        self.replace_nonascii = replace_nonascii
        self.tokenize = tokenize
        self.remove_stopwords = remove_stopwords
        self.lemmatization = lemmatization
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed = []
        for doc in X:
            if self.lower_case:
                doc = doc.lower()
            if self.replace_usernames:
                doc = re.sub(r'@([^\s]+)',' USERNAME ', doc)
            if self.unescape_html:
                doc = unescape(doc)
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(doc)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    doc = doc.replace(url, ' URL ')
            if self.replace_numbers:
                doc = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', ' NUMBER ', doc)
            if self.remove_junk:
                pattern = r'\¥|\â|\«|\»|\Ñ|\Ð|\¼|\½|\¾|\!|\?|\¿|\x82\
                            |\x83|\x84|\x85|\x86|\x87|\x88|\x89|\
                            |\x8a|\x8b|\x8c|\x8d|\x8e|\°|\µ|\´|\º|\¹|\³'
                doc = re.sub(pattern,'', doc)
            if self.remove_punctuation:
                doc = re.sub(r'\W+', ' ', doc, flags=re.M)
            if self.replace_emojis:
                doc = re.sub(r'[^\x00-\x7F]+', ' EMOJI ', doc)
            if self.replace_nonascii:
                if is_ascii(doc) == False:
                    doc = ' NONASCII '
            word_counts = Counter(doc.split())
            if self.remove_stopwords:
                #25 semantically non-selective words from the Reuters-RCV1 dataset
                # plus single-digit letters
                stop_words = ['a','an','and','are','as','at','be','by','for','from',
                              'has','he','in','is','it','its','of','on','that','the',
                              'to','was','were','will','with','t','s','d','m']
                for word in stop_words:
                    try:
                        word_counts.pop(word)
                    except KeyError:
                        continue
            if self.lemmatization and lemmatizer is not None:
                lemmatized_word_counts = Counter()
                for word, count in word_counts.items():
                    lemmatized_word = lemmatizer.lemmatize(word)
                    lemmatized_word_counts[lemmatized_word] += count
                word_counts = lemmatized_word_counts      
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [7]:
X_wordcounts = DocumentToWordCounterTransformer(tokenize=False).fit_transform(X_array)

In [8]:
len(X_wordcounts)

119747

In [9]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.most_common_ = most_common
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [10]:
vocab_transformer = WordCounterToVectorTransformer()
X_vectors = vocab_transformer.fit_transform(X_wordcounts)
X_vectors

<119747x1001 sparse matrix of type '<class 'numpy.int32'>'
	with 1040315 stored elements in Compressed Sparse Row format>

In [11]:
len(vocab_transformer.vocabulary_)

1000

In [12]:
for k,v in vocab_transformer.vocabulary_.items():
    if v < 11:
        print(v, k)

1 i
2 USERNAME
3 NUMBER
4 my
5 you
6 me
7 so
8 have
9 but
10 just


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

preprocess_pipeline = Pipeline([
    ("document_to_wordcount", DocumentToWordCounterTransformer()),
    ("wordcount_to_vector", WordCounterToVectorTransformer()),
])

X_train_transformed = preprocess_pipeline.fit_transform(X_array)

In [14]:
X_train_transformed

<119747x1001 sparse matrix of type '<class 'numpy.int32'>'
	with 1040315 stored elements in Compressed Sparse Row format>

In [15]:
y_array = y.iloc[:,0].ravel()

In [16]:
len(y_array)

119747

In [17]:
from sklearn.model_selection import cross_val_score

log_clf = LogisticRegression(solver="liblinear", random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_array, cv=10, verbose=3, scoring='accuracy')
print('Mean accuracy: ' + str(score.mean()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] .................................... , score=0.766, total=   1.6s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV] .................................... , score=0.759, total=   1.7s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.3s remaining:    0.0s


[CV] .................................... , score=0.766, total=   1.6s
[CV]  ................................................................
[CV] .................................... , score=0.762, total=   1.7s
[CV]  ................................................................
[CV] .................................... , score=0.767, total=   1.7s
[CV]  ................................................................
[CV] .................................... , score=0.762, total=   1.6s
[CV]  ................................................................
[CV] .................................... , score=0.766, total=   1.5s
[CV]  ................................................................
[CV] .................................... , score=0.763, total=   1.7s
[CV]  ................................................................
[CV] .................................... , score=0.757, total=   1.7s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   16.4s finished


In [18]:
from sklearn.naive_bayes import MultinomialNB

NB_clf = MultinomialNB()
score = cross_val_score(NB_clf, X_train_transformed, y_array, cv=10, verbose=3, scoring='accuracy')
print('Mean accuracy: ' + str(score.mean()))

[CV]  ................................................................
[CV] .................................... , score=0.755, total=   0.1s
[CV]  ................................................................
[CV] .................................... , score=0.749, total=   0.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .................................... , score=0.755, total=   0.1s
[CV]  ................................................................
[CV] .................................... , score=0.751, total=   0.1s
[CV]  ................................................................
[CV] .................................... , score=0.754, total=   0.1s
[CV]  ................................................................
[CV] .................................... , score=0.750, total=   0.1s
[CV]  ................................................................
[CV] .................................... , score=0.754, total=   0.1s
[CV]  ................................................................
[CV] .................................... , score=0.747, total=   0.1s
[CV]  ................................................................
[CV] .................................... , score=0.745, total=   0.1s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.6s finished


In [19]:
# time notebook
mins, secs = divmod(time.time() - start_notebook, 60)
print(f'Total running time: {mins:0.0f} minute(s) and {secs:0.0f} second(s).')

Total running time: 1 minute(s) and 47 second(s).


---